In [1]:
import pandas as pd
import numpy as np
import yfinance as yahooFinance
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
START_DATE = '2023-08-01'
END_DATE = '2024-08-01'
TICKERS2CALLDATE = {
    "NVDA": "2024-05-22",
    "TSM": "2024-04-18",
    "AAPL": "2024-05-02",
    "TSLA": "2024-04-23",
    "MSFT": "2024-04-25",
    "JPM": "2024-04-12",
    "V": "2024-04-24"
}
EVENT_WINDOW = [2, 3]

In [3]:
def get_ticker_df(ticker):
    res_df = yahooFinance.Ticker(ticker).history(start=START_DATE, end=END_DATE)
    return res_df

def get_returns(prices):
    returns = []
    for i in range(1, len(prices)):
        returns.append(prices[i] / prices[i-1] - 1)
    returns.append(0)
    return np.array(returns)

def get_AR_CMR(returns):
    """
    returns
    model_returns: array of returns by the model
    ar_returns: array of abnormal returns
    ar_std: standard deviation of the abnormal returns
    mva_ar_returns: array of moving average of abnormal returns
    """
    model_returns = np.mean(returns) * np.ones(len(returns))
    ar_returns = returns - model_returns
    ar_std = np.std(ar_returns)
    mva_ar_returns = np.convolve(ar_returns, np.ones(EVENT_WINDOW[0] + EVENT_WINDOW[1]) / (EVENT_WINDOW[0] + EVENT_WINDOW[1]), mode='valid')
    return model_returns, ar_std, ar_returns, mva_ar_returns

def get_AR_CAPM(returns, market_returns):
    """
    returns
    model_returns: array of returns by the model
    ar_returns: array of abnormal returns
    mva_ar_returns: array of moving average of abnormal returns
    """
    beta = np.cov(returns, market_returns)[0][1] / np.var(market_returns)
    model_returns = beta * market_returns + np.mean(returns - beta * market_returns)
    ar_returns = returns - model_returns
    ar_std = np.std(ar_returns)
    mva_ar_returns = np.convolve(ar_returns, np.ones(EVENT_WINDOW[0] + EVENT_WINDOW[1]) / (EVENT_WINDOW[0] + EVENT_WINDOW[1]), mode='valid')
    return model_returns, ar_returns, ar_std, mva_ar_returns

def _get_FF_coeff_df():
    ff_coeff_df = pd.read_csv("../data/F-F_Research_Data_Factors_daily.csv")
    ff_coeff_df["date"] = pd.to_datetime(ff_coeff_df["date"], format='%Y%m%d')
    ff_coeff_df = ff_coeff_df[(ff_coeff_df["date"] >= START_DATE) & (ff_coeff_df["date"] <= END_DATE)]
    ff_coeff_df = ff_coeff_df.set_index("date")
    return ff_coeff_df

def get_AR_FF(returns, market_returns):
    """
    returns
    model_returns: array of returns by the model
    model_std: standard deviation of the model returns
    ar_returns: array of abnormal returns
    mva_ar_returns: array of moving average of abnormal returns
    """
    ff_coeff_df = _get_FF_coeff_df()
    b1 = np.cov(returns - ff_coeff_df["RF"], market_returns - ff_coeff_df["RF"])[0][1] / np.var(market_returns - ff_coeff_df["RF"])
    b2 = np.cov(returns, ff_coeff_df["SMB"])[0][1] / np.var(ff_coeff_df["SMB"])
    b3 = np.cov(returns, ff_coeff_df["HML"])[0][1] / np.var(ff_coeff_df["HML"])
    alpha = np.mean(returns - ff_coeff_df["RF"] - b1 * (market_returns - ff_coeff_df["RF"]) - b2 * ff_coeff_df["SMB"] - b3 * ff_coeff_df["HML"])
    model_returns = alpha + ff_coeff_df["RF"] + b1 * (market_returns - ff_coeff_df["RF"]) + b2 * ff_coeff_df["SMB"] + b3 * ff_coeff_df["HML"]
    ar_returns = returns - model_returns
    ar_std = np.std(ar_returns)
    mva_ar_returns = np.convolve(ar_returns, np.ones(EVENT_WINDOW[0] + EVENT_WINDOW[1]) / (EVENT_WINDOW[0] + EVENT_WINDOW[1]), mode='valid')
    return model_returns, ar_returns, ar_std, mva_ar_returns

def plot_model_returns(ticker, prices, returns, model_returns):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
    fig.add_trace(go.Scatter(x=prices.index, y=prices["Close"], name="Close Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=prices.index, y=returns, name="Returns"), row=2, col=1)
    fig.add_trace(go.Scatter(x=prices.index, y=model_returns, name="Model Returns"), row=2, col=1)
    fig.update_layout(title=f"{ticker} Model Returns", xaxis_title="Date", yaxis_title="Price", showlegend=True)
    fig.show()

In [4]:
def plot_MR_CMR(ticker):
    prices = get_ticker_df(ticker)
    returns = get_returns(prices["Close"].values)
    model_returns, ar_std, ar_returns, mva_ar_returns = get_AR_CMR(returns)
    plot_model_returns(ticker, prices, returns, model_returns)

def plot_MR_CAPM(ticker):
    prices = get_ticker_df(ticker)
    returns = get_returns(prices["Close"].values)
    market_prices = get_ticker_df("^GSPC")
    market_returns = get_returns(market_prices["Close"].values)
    model_returns, ar_returns, ar_std, mva_ar_returns = get_AR_CAPM(returns, market_returns)
    plot_model_returns(ticker, prices, returns, model_returns)

def plot_MR_FF(ticker):
    prices = get_ticker_df(ticker)
    returns = get_returns(prices["Close"].values)
    market_prices = get_ticker_df("^GSPC")
    market_returns = get_returns(market_prices["Close"].values)
    model_returns, ar_returns, ar_std, mva_ar_returns = get_AR_FF(returns, market_returns)
    plot_model_returns(ticker, prices, returns, model_returns)

In [5]:
def get_MSE_R2_AR(ticker, model):
    prices = get_ticker_df(ticker)
    returns = get_returns(prices["Close"].values)
    market_prices = get_ticker_df("^GSPC")
    market_returns = get_returns(market_prices["Close"].values)
    if model == "CMR":
        model_returns, ar_std, ar_returns, mva_ar_returns = get_AR_CMR(returns)
    elif model == "CAPM":
        model_returns, ar_returns, ar_std, mva_ar_returns = get_AR_CAPM(returns, market_returns)
    elif model == "FF":
        model_returns, ar_returns, ar_std, mva_ar_returns = get_AR_FF(returns, market_returns)
    mse = np.mean(ar_returns ** 2)
    r2 = 1 - mse / np.var(returns)
    return mse, r2

# results = {}
# for ticker, call_date in TICKERS2CALLDATE.items():
#     for model in ["CMR", "CAPM", "FF"]:
#         mse, r2 = get_MSE_R2_AR(ticker, model)
#         results[(ticker, model)] = [mse, r2]

# results_df = pd.DataFrame(results, index=["MSE", "R2"]).T
# results_df

In [6]:
results_df = pd.read_csv("../data/model_comparison.csv")
results_df = results_df.rename(columns={"Unnamed: 0": "Ticker", "Unnamed: 1": "Model"})
results_df.set_index(["Ticker", "Model"])

MSE        R2
Ticker Model                    
NVDA   CMR    0.000942  0.000000
       CAPM   0.000609  0.353237
       FF     0.000603  0.359907
TSM    CMR    0.000472  0.000000
       CAPM   0.000288  0.389231
       FF     0.000284  0.397069
AAPL   CMR    0.000204  0.000000
       CAPM   0.000138  0.319751
       FF     0.000136  0.329650
TSLA   CMR    0.001131  0.000000
       CAPM   0.000894  0.209507
       FF     0.000880  0.221990
MSFT   CMR    0.000154  0.000000
       CAPM   0.000080  0.479133
       FF     0.000079  0.483250
JPM    CMR    0.000121  0.000000
       CAPM   0.000103  0.152938
       FF     0.000102  0.154704
V      CMR    0.000084  0.000000
       CAPM   0.000058  0.310254
       FF     0.000058  0.310912

In [13]:
# abnormal returns
from scipy import stats
def plot_AR(ticker):
    prices = get_ticker_df(ticker)
    returns = get_returns(prices["Close"].values)
    market_prices = get_ticker_df("^GSPC")
    market_returns = get_returns(market_prices["Close"].values)
    model_returns, ar_returns, ar_std, mva_ar_returns = get_AR_FF(returns, market_returns)
    earning_date = prices.index[prices.index.date == pd.to_datetime(TICKERS2CALLDATE[ticker]).date()][0]
    earning_index = prices.index.get_loc(earning_date)
    z_score = mva_ar_returns[earning_index] / ar_std / np.sqrt(EVENT_WINDOW[0] + EVENT_WINDOW[1])
    p_value = stats.norm.cdf(z_score)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=prices.index, y=ar_returns, name="AR"))
    fig.add_vline(x=earning_date, line_dash="dash", line_color="black")
    fig.add_trace(go.Scatter(x=prices.index[EVENT_WINDOW[0]:-EVENT_WINDOW[1]], y=mva_ar_returns, name="MVA AR"))
    fig.add_hrect(y0=-ar_std, y1=ar_std, fillcolor="#F0E68C", opacity=0.2, layer="below", line_width=0)
    fig.update_layout(
        title=f"{ticker} Abnormal Returns",
        xaxis_title="Date",
        yaxis_title="Abnormal Returns",
        showlegend=True,
        colorway=["lightgrey", "red", "black"]
        )


    return fig, z_score, p_value

In [14]:
p_z = {}
for ticker in TICKERS2CALLDATE.keys():
    fig, z_score, p_value = plot_AR(ticker)
    fig.show()
    p_z[ticker] = [p_value, z_score]

p_z_df = pd.DataFrame(p_z, index=["p-value", "z-score"]).T

In [15]:
p_z_df

,p-value,z-score
NVDA,0.743027,0.652705
TSM,0.536312,0.091147
AAPL,0.601163,0.256358
TSLA,0.794458,0.821986
MSFT,0.459930,-0.100611
JPM,0.608506,0.275427
V,0.400215,-0.252791
